# TennerGrid

- Write numbers from 0 to 9 into the empty cells of the grid
- so that each number occurs exactly once in each row.
- Same numbers must not be orthogonally or diagonally adjacent.
- The sum of the numbers in each column must match the number in the last row.

---------


![](https://www.janko.at/Raetsel/Zehnergitter/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Zehnergitter/Regeln-02.gif)

> https://www.janko.at/Raetsel/Zehnergitter/index.htm

In [1]:
def readGrid(path):
    with open(f"../assets/data/TennerGrid/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n),  res

if __name__ == "__main__":
    m, n, grid = readGrid("1_6x10")
    print(m, n)
    # print(grid)
    for g in grid:
        print(g)
    

6 10
['-', '4', '-', '2', '-', '9', '3', '8', '7', '5']
['5', '-', '1', '7', '0', '8', '6', '9', '3', '4']
['4', '-', '5', '-', '3', '7', '-', '-', '-', '-']
['2', '6', '4', '8', '5', '-', '-', '7', '-', '-']
['-', '3', '-', '-', '-', '-', '5', '-', '8', '6']
['12', '23', '12', '20', '18', '40', '17', '34', '24', '25']


In [3]:
from z3 import Solver, Int, Distinct, sat, And, Or, If, Sum

def tennerGrid_solver(m, n, grid):
    x = [[Int(f"x_{i}_{j}") for j in range(n)] for i in range(m - 1)]
    s = Solver()

    for i in range(m - 1):
        for j in range(n):
            if grid[i][j].isdigit():
                s.add(x[i][j] == int(grid[i][j]))

    # define the existing number
    cells_constr  = [ And(0 <= x[i][j], x[i][j] <= 9) 
             for i in range(m - 1) for j in range(n) ]
    # define the range of variables
    
    for i in range(m - 1):
        s.add(Distinct([x[i][j] for j in range(n)]))

    for j in range(n):
        s.add(Sum([x[i][j] for i in range(m - 1)]) == int(grid[m - 1][j]))
    # add summation constr ... 
    
    for i in range(m - 1):
        for j in range(n):
            sur = []
            for i1 in range(-1, 2, 1):
                for j1 in range(-1, 2, 1):
                    if (i1 == 0 and j1 == 0) or (i + i1 < 0 or i + i1 >= m - 1) or (j + j1 < 0 or j + j1 >= n) :
                        continue
                    sur.append(x[i + i1][j + j1])
            for k in range(len(sur)):
                s.add(x[i][j] != sur[k])
    
    s.add(cells_constr)
    if s.check() == sat:
        model = s.model()
        r = [ [ model.evaluate(x[i][j]) for j in range(n) ] 
              for i in range(m - 1) ]
        for i in range(m - 1):
            for j in range(n):
                print(r[i][j], end = " ")
            print("")
        for j in range(n):
            print(int(grid[m - 1][j]), end = " ")

if __name__ == "__main__":
    m, n, grid = readGrid("1_6x10")
    tennerGrid_solver(m, n, grid)

1 4 0 2 6 9 3 8 7 5 
5 2 1 7 0 8 6 9 3 4 
4 8 5 2 3 7 0 1 6 9 
2 6 4 8 5 9 3 7 0 1 
0 3 2 1 4 7 5 9 8 6 
12 23 12 20 18 40 17 34 24 25 

In [2]:
from ortools.sat.python import cp_model

def tennerGrid_solver_ortools(m, n, grid):
    # 1. 创建模型
    model = cp_model.CpModel()

    # 网格数据所在的行数（最后一行是求和目标，不参与填数）
    grid_rows = m - 1
    grid_cols = n

    # 2. 定义变量
    # x[i][j] 对应网格第 i 行 第 j 列的数字，范围 0-9
    x = {}
    for i in range(grid_rows):
        for j in range(grid_cols):
            x[(i, j)] = model.NewIntVar(0, 9, f"x_{i}_{j}")

    # 3. 添加约束

    # (A) 预填数字约束 (Pre-filled values)
    for i in range(grid_rows):
        for j in range(grid_cols):
            cell_content = grid[i][j]
            # 检查内容是否为数字（排除占位符或空字符）
            if cell_content.strip().isdigit():
                model.Add(x[(i, j)] == int(cell_content))

    # (B) 行约束：每行数字必须互不相同 (Distinct row)
    for i in range(grid_rows):
        row_vars = [x[(i, j)] for j in range(grid_cols)]
        model.AddAllDifferent(row_vars)

    # (C) 列求和约束：每列之和等于最后一行给出的数字 (Column Sum)
    target_sums = grid[m - 1] # 最后一行是目标值
    for j in range(grid_cols):
        col_vars = [x[(i, j)] for i in range(grid_rows)]
        target_val = int(target_sums[j])
        model.Add(sum(col_vars) == target_val)

    # (D) 相邻约束：正交和对角线相邻的格子不能相同 (Adjacency)
    # 为了避免重复添加约束（比如 A!=B 和 B!=A），我们只需要检查“后方”的邻居即可。
    # 这些方向包括：右(0,1), 下(1,0), 左下(1,-1), 右下(1,1)
    directions = [(0, 1), (1, 0), (1, -1), (1, 1)]
    
    for i in range(grid_rows):
        for j in range(grid_cols):
            for di, dj in directions:
                ni, nj = i + di, j + dj
                # 检查边界
                if 0 <= ni < grid_rows and 0 <= nj < grid_cols:
                    model.Add(x[(i, j)] != x[(ni, nj)])

    # 4. 求解
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # 5. 输出结果
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Solution found (Status: {solver.StatusName(status)}):")
        for i in range(grid_rows):
            row_vals = []
            for j in range(grid_cols):
                val = solver.Value(x[(i, j)])
                row_vals.append(str(val))
            print(" ".join(row_vals))
        
        # 打印目标和（作为参考）
        print("-" * (grid_cols * 2))
        print(" ".join([str(s) for s in target_sums]))
    else:
        print("No solution found.")

# --- 模拟数据读取部分 (为了让代码可直接运行，我模拟了 readGrid 的行为) ---
# 如果你有实际文件，请替换回你的 readGrid 函数
def mock_run():
    # 这是一个典型的 3行填数 + 1行求和 的 TennerGrid 示例
    # 假设输入类似：
    # -1 -1 3 -1 -1
    # -1 -1 -1 2 -1
    # -1 5 -1 -1 -1
    # 12 15 10 16 9
    
    m_input = 4 # 3行数据 + 1行和
    n_input = 5
    
    # 模拟 grid 数据，'-1' 代表空
    grid_input = [
        ['-1', '-1', '3', '-1', '-1'],
        ['-1', '-1', '-1', '2', '-1'],
        ['-1', '5', '-1', '-1', '-1'],
        ['12', '15', '10', '16', '9'] # sum row
    ]
    
    print(f"Solving Grid: {m_input}x{n_input}")
    tennerGrid_solver_ortools(m_input, n_input, grid_input)

if __name__ == "__main__":
    # 如果你想用文件的 readGrid，请取消注释下面几行并注释掉 mock_run
    m, n, grid = readGrid("1_6x10")
    tennerGrid_solver_ortools(m, n, grid)
    
    # mock_run()

Solution found (Status: OPTIMAL):
1 4 0 2 6 9 3 8 7 5
5 2 1 7 0 8 6 9 3 4
4 8 5 2 3 7 0 1 6 9
2 6 4 8 5 9 3 7 0 1
0 3 2 1 4 7 5 9 8 6
--------------------
12 23 12 20 18 40 17 34 24 25
